In [ ]:
from plotly.subplots import make_subplots
import plotly.io as pio
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo dos gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pio.templates.default = "plotly_white"

print("📊 Notebook de Análise - Sistema de Pâncreas Artificial")
print("="*60)
print("⚠️  SISTEMA EXPERIMENTAL - APENAS PARA PESQUISA")
print("="*60)

# Célula 2: Função para carregar dados
def load_simulation_data(csv_files=None):
    """
    Carrega dados de simulação de arquivos CSV ou gera dados sintéticos
    
    Args:
        csv_files: Lista de arquivos CSV para carregar
        
    Returns:
        DataFrame com dados de simulação
    """
    if csv_files is None:
        print("📄 Gerando dados sintéticos para demonstração...")
        return generate_synthetic_data()
    
    dfs = []
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            dfs.append(df)
            print(f"✅ Carregado: {file} ({len(df)} registros)")
        except FileNotFoundError:
            print(f"❌ Arquivo não encontrado: {file}")
        except Exception as e:
            print(f"❌ Erro ao carregar {file}: {e}")
    
    if dfs:
        combined_df = pd.concat(dfs, ignore_index=True)
        combined_df = combined_df.sort_values('timestamp').reset_index(drop=True)
        return combined_df
    else:
        return generate_synthetic_data()

def generate_synthetic_data(days=7):
    """Gera dados sintéticos realistas para demonstração"""
    
    # Parâmetros da simulação
    start_time = datetime.now() - timedelta(days=days)
    timestep_minutes = 10
    total_points = days * 24 * 6  # 6 pontos por hora
    
    data = []
    
    for i in range(total_points):
        timestamp = start_time + timedelta(minutes=i * timestep_minutes)
        hour = timestamp.hour
        day_of_week = timestamp.weekday()
        
        # Modelo circadiano de glicemia
        circadian_base = 120
        dawn_effect = 15 * np.sin(2 * np.pi * (hour - 6) / 24)  # Dawn phenomenon
        weekly_variation = 5 * np.sin(2 * np.pi * day_of_week / 7)
        
        # Ruído e variabilidade individual
        noise = np.random.normal(0, 12)
        
        # Eventos de refeição (probabilísticos)
        meal_effect = 0
        if hour in [7, 12, 19] and np.random.random() < 0.7:  # Refeições principais
            meal_magnitude = np.random.uniform(30, 80)
            meal_duration = np.random.uniform(90, 180)  # minutos
            time_since_meal = (i % (meal_duration // timestep_minutes)) * timestep_minutes
            
            if time_since_meal <= meal_duration:
                # Curva de absorção tipo log-normal
                peak_time = meal_duration * 0.3
                meal_effect = meal_magnitude * np.exp(-(np.log(time_since_meal + 1) - np.log(peak_time))**2 / (2 * 0.5**2))
        
        # Efeito de exercício (ocasional)
        exercise_effect = 0
        if hour in [17, 18, 19] and np.random.random() < 0.2:  # Exercício à tarde
            exercise_effect = -np.random.uniform(20, 40)
        
        # Glicemia final
        glucose = max(60, min(350, 
            circadian_base + dawn_effect + weekly_variation + 
            meal_effect + exercise_effect + noise
        ))
        
        # Calcular dose de insulina baseada em controlador simples
        target_glucose = 120
        error = glucose - target_glucose
        
        # PID simplificado
        kp = 0.05
        insulin_dose = max(0, kp * error / 100)  # Converter para unidades apropriadas
        
        # Adicionar ruído na dose
        if insulin_dose > 0:
            insulin_dose += np.random.normal(0, 0.1)
            insulin_dose = max(0, insulin_dose)
        
        # Determinar tipo de dose
        if meal_effect > 20:
            dose_type = 'bolus'
        elif insulin_dose > 0.5:
            dose_type = 'correction'
        else:
            dose_type = 'basal'
        
        # Simular confiança do sistema
        confidence = min(1.0, 0.5 + (days - abs(i / (total_points / days) - days/2)) / days)
        confidence += np.random.normal(0, 0.1)
        confidence = max(0.1, min(0.95, confidence))
        
        # Simular componentes PID e ML
        pid_component = insulin_dose * 0.8
        ml_component = insulin_dose * 0.2 * confidence
        
        data.append({
            'timestamp': timestamp,
            'step': i,
            'glucose': glucose,
            'insulin_dose': insulin_dose,
            'dose_type': dose_type,
            'confidence': confidence,
            'pid_component': pid_component,
            'ml_component': ml_component,
            'hour_of_day': hour,
            'day_of_week': day_of_week,
            'meal_effect': meal_effect,
            'exercise_effect': exercise_effect
        })
    
    df = pd.DataFrame(data)
    print(f"📊 Dados sintéticos gerados: {len(df)} pontos ({days} dias)")
    
    return df

# Célula 3: Carregar dados
# Exemplo de uso - substituir pelos seus arquivos CSV reais
csv_files = [
    # 'exports/simulation_data_20241210.csv',
    # 'exports/controller_decisions_20241210.csv'
]

df = load_simulation_data(csv_files)
print(f"\n📈 Dataset carregado: {len(df)} registros")
print(f"📅 Período: {df['timestamp'].min()} até {df['timestamp'].max()}")
print(f"⏱️ Duração: {(df['timestamp'].max() - df['timestamp'].min()).days} dias")

# Célula 4: Análise exploratória básica
def basic_statistics(df):
    """Calcula estatísticas básicas do dataset"""
    
    print("📊 ESTATÍSTICAS BÁSICAS")
    print("="*40)
    
    # Estatísticas de glicemia
    glucose_stats = df['glucose'].describe()
    print(f"Glicemia (mg/dL):")
    print(f"  Média: {glucose_stats['mean']:.1f}")
    print(f"  Mediana: {glucose_stats['50%']:.1f}")
    print(f"  Desvio padrão: {glucose_stats['std']:.1f}")
    print(f"  Min: {glucose_stats['min']:.1f}")
    print(f"  Max: {glucose_stats['max']:.1f}")
    
    # Tempo em faixa
    in_range = ((df['glucose'] >= 70) & (df['glucose'] <= 180)).sum()
    below_range = (df['glucose'] < 70).sum()
    above_range = (df['glucose'] > 180).sum()
    total = len(df)
    
    print(f"\nTempo em Faixa:")
    print(f"  70-180 mg/dL: {in_range/total*100:.1f}%")
    print(f"  < 70 mg/dL: {below_range/total*100:.1f}%")
    print(f"  > 180 mg/dL: {above_range/total*100:.1f}%")
    
    # Estatísticas de insulina
    insulin_stats = df['insulin_dose'].describe()
    print(f"\nInsulina (U):")
    print(f"  Total diário médio: {insulin_stats['mean']*24*6:.1f}")
    print(f"  Dose máxima: {insulin_stats['max']:.2f}")
    print(f"  Doses > 0: {(df['insulin_dose'] > 0).sum()}")
    
    return {
        'glucose_mean': glucose_stats['mean'],
        'glucose_std': glucose_stats['std'],
        'time_in_range': in_range/total,
        'time_below': below_range/total,
        'time_above': above_range/total,
        'daily_insulin': insulin_stats['mean']*24*6
    }

stats = basic_statistics(df)

# Célula 5: Visualizações principais
def create_glucose_timeline(df, days_to_show=3):
    """Cria gráfico de timeline de glicemia interativo"""
    
    # Filtrar últimos N dias para visualização
    end_date = df['timestamp'].max()
    start_date = end_date - timedelta(days=days_to_show)
    df_filtered = df[df['timestamp'] >= start_date].copy()
    
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=['Glicemia (mg/dL)', 'Insulina (U)', 'Confiança do Sistema'],
        vertical_spacing=0.08,
        row_heights=[0.5, 0.25, 0.25]
    )
    
    # Gráfico principal de glicemia
    fig.add_trace(
        go.Scatter(
            x=df_filtered['timestamp'],
            y=df_filtered['glucose'],
            mode='lines',
            name='Glicemia',
            line=dict(color='blue', width=2),
            hovertemplate='%{x}<br>Glicemia: %{y:.1f} mg/dL<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Zonas de referência
    fig.add_hline(y=70, line_dash="dash", line_color="orange", 
                  annotation_text="Limite inferior (70)", row=1, col=1)
    fig.add_hline(y=180, line_dash="dash", line_color="red", 
                  annotation_text="Limite superior (180)", row=1, col=1)
    fig.add_hline(y=120, line_dash="dot", line_color="green", 
                  annotation_text="Alvo (120)", row=1, col=1)
    
    # Gráfico de insulina
    colors = {'basal': 'lightblue', 'bolus': 'orange', 'correction': 'red'}
    for dose_type in df_filtered['dose_type'].unique():
        mask = df_filtered['dose_type'] == dose_type
        fig.add_trace(
            go.Scatter(
                x=df_filtered[mask]['timestamp'],
                y=df_filtered[mask]['insulin_dose'],
                mode='markers',
                name=f'Insulina {dose_type}',
                marker=dict(color=colors.get(dose_type, 'gray'), size=6),
                hovertemplate=f'{dose_type}: %{{y:.2f}}U<br>%{{x}}<extra></extra>'
            ),
            row=2, col=1
        )
    
    # Gráfico de confiança
    fig.add_trace(
        go.Scatter(
            x=df_filtered['timestamp'],
            y=df_filtered['confidence'],
            mode='lines',
            name='Confiança',
            line=dict(color='purple', width=2),
            fill='tonexty',
            hovertemplate='Confiança: %{y:.1%}<extra></extra>'
        ),
        row=3, col=1
    )
    
    fig.update_layout(
        title=f"Timeline do Sistema APS - Últimos {days_to_show} dias",
        height=800,
        showlegend=True,
        hovermode='x unified'
    )
    
    # Configurar eixos
    fig.update_yaxes(title_text="mg/dL", row=1, col=1, range=[50, 300])
    fig.update_yaxes(title_text="Unidades", row=2, col=1)
    fig.update_yaxes(title_text="Confiança", row=3, col=1, range=[0, 1], tickformat='.0%')
    fig.update_xaxes(title_text="Data/Hora", row=3, col=1)
    
    return fig

# Criar e exibir gráfico
fig_timeline = create_glucose_timeline(df, days_to_show=3)
fig_timeline.show()

# Célula 6: Análise de padrões circadianos
def analyze_circadian_patterns(df):
    """Analisa padrões circadianos na glicemia"""
    
    # Agrupar por hora do dia
    hourly_stats = df.groupby('hour_of_day').agg({
        'glucose': ['mean', 'std', 'count'],
        'insulin_dose': 'sum'
    }).round(2)
    
    hourly_stats.columns = ['glucose_mean', 'glucose_std', 'count', 'total_insulin']
    hourly_stats = hourly_stats.reset_index()
    
    # Criar gráfico circadiano
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=['Glicemia Média por Hora', 'Insulina Total por Hora'],
        vertical_spacing=0.15
    )
    
    # Glicemia média com banda de erro
    fig.add_trace(
        go.Scatter(
            x=hourly_stats['hour_of_day'],
            y=hourly_stats['glucose_mean'],
            mode='lines+markers',
            name='Glicemia média',
            line=dict(color='blue', width=3),
            error_y=dict(
                type='data',
                array=hourly_stats['glucose_std'],
                visible=True,
                color='lightblue'
            )
        ),
        row=1, col=1
    )
    
    # Insulina total por hora
    fig.add_trace(
        go.Bar(
            x=hourly_stats['hour_of_day'],
            y=hourly_stats['total_insulin'],
            name='Insulina total',
            marker_color='orange',
            opacity=0.7
        ),
        row=2, col=1
    )
    
    fig.update_layout(
        title="Padrões Circadianos",
        height=600,
        showlegend=True
    )
    
    fig.update_xaxes(title_text="Hora do Dia", dtick=2)
    fig.update_yaxes(title_text="Glicemia (mg/dL)", row=1, col=1)
    fig.update_yaxes(title_text="Insulina Total (U)", row=2, col=1)
    
    return fig, hourly_stats

fig_circadian, hourly_data = analyze_circadian_patterns(df)
fig_circadian.show()

# Identificar padrões
dawn_effect = hourly_data[hourly_data['hour_of_day'].between(4, 8)]['glucose_mean'].max() - \
              hourly_data[hourly_data['hour_of_day'].between(4, 8)]['glucose_mean'].min()

print(f"📈 Dawn Phenomenon detectado: {dawn_effect:.1f} mg/dL de variação matinal")

# Célula 7: Análise de eficácia do controlador
def analyze_controller_performance(df):
    """Analisa desempenho do controlador híbrido"""
    
    # Calcular correlação entre componentes
    corr_pid_ml = df['pid_component'].corr(df['ml_component'])
    
    # Analisar evolução da confiança ao longo do tempo
    df['days_from_start'] = (df['timestamp'] - df['timestamp'].min()).dt.total_seconds() / (24 * 3600)
    
    # Regressão linear da confiança ao longo do tempo
    from scipy import stats
    slope, intercept, r_value, p_value, std_err = stats.linregress(df['days_from_start'], df['confidence'])
    
    print("🤖 ANÁLISE DO CONTROLADOR")
    print("="*30)
    print(f"Correlação PID-ML: {corr_pid_ml:.3f}")
    print(f"Evolução da confiança: {slope:.3f}/dia (R²={r_value**2:.3f})")
    
    # Gráfico de evolução da confiança
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=df['timestamp'],
            y=df['confidence'],
            mode='markers',
            name='Confiança',
            marker=dict(color='purple', size=4, opacity=0.6)
        )
    )
    
    # Linha de tendência
    trend_y = slope * df['days_from_start'] + intercept
    fig.add_trace(
        go.Scatter(
            x=df['timestamp'],
            y=trend_y,
            mode='lines',
            name='Tendência',
            line=dict(color='red', width=2, dash='dash')
        )
    )
    
    fig.update_layout(
        title="Evolução da Confiança do Sistema",
        xaxis_title="Tempo",
        yaxis_title="Confiança",
        height=400
    )
    
    fig.update_yaxes(tickformat='.0%')
    
    return fig

fig_confidence = analyze_controller_performance(df)
fig_confidence.show()

# Célula 8: Detecção de eventos adversos
def detect_adverse_events(df):
    """Detecta e analisa eventos adversos"""
    
    events = []
    
    # Detectar hipoglicemias (< 70 mg/dL por > 15 min)
    hypo_mask = df['glucose'] < 70
    
    # Detectar sequências contínuas
    hypo_sequences = []
    in_hypo = False
    start_idx = None
    
    for i, is_hypo in enumerate(hypo_mask):
        if is_hypo and not in_hypo:
            in_hypo = True
            start_idx = i
        elif not is_hypo and in_hypo:
            in_hypo = False
            duration_min = (i - start_idx) * 10  # 10 min por ponto
            if duration_min >= 15:
                hypo_sequences.append({
                    'start': start_idx,
                    'end': i-1,
                    'duration_min': duration_min,
                    'min_glucose': df.iloc[start_idx:i]['glucose'].min()
                })
    
    # Detectar hiperglicemias prolongadas (> 250 mg/dL por > 2h)
    hyper_mask = df['glucose'] > 250
    
    hyper_sequences = []
    in_hyper = False
    start_idx = None
    
    for i, is_hyper in enumerate(hyper_mask):
        if is_hyper and not in_hyper:
            in_hyper = True
            start_idx = i
        elif not is_hyper and in_hyper:
            in_hyper = False
            duration_min = (i - start_idx) * 10
            if duration_min >= 120:  # 2 horas
                hyper_sequences.append({
                    'start': start_idx,
                    'end': i-1,
                    'duration_min': duration_min,
                    'max_glucose': df.iloc[start_idx:i]['glucose'].max()
                })
    
    print("🚨 EVENTOS ADVERSOS DETECTADOS")
    print("="*35)
    print(f"Hipoglicemias (≥15 min): {len(hypo_sequences)}")
    for i, event in enumerate(hypo_sequences[:5], 1):  # Mostrar até 5
        timestamp = df.iloc[event['start']]['timestamp']
        print(f"  {i}. {timestamp}: {event['duration_min']} min, mín {event['min_glucose']:.1f} mg/dL")
    
    print(f"\nHiperglicemias (≥2h): {len(hyper_sequences)}")
    for i, event in enumerate(hyper_sequences[:5], 1):
        timestamp = df.iloc[event['start']]['timestamp']
        print(f"  {i}. {timestamp}: {event['duration_min']} min, máx {event['max_glucose']:.1f} mg/dL")
    
    return hypo_sequences, hyper_sequences

hypo_events, hyper_events = detect_adverse_events(df)

# Célula 9: Métricas avançadas de qualidade
def calculate_advanced_metrics(df):
    """Calcula métricas avançadas de qualidade"""
    
    glucose_values = df['glucose'].values
    
    # GMI (Glucose Management Indicator) - estimativa de HbA1c
    mean_glucose = np.mean(glucose_values)
    gmi = 3.31 + 0.02392 * mean_glucose
    
    # Coeficiente de Variação
    cv = (np.std(glucose_values) / mean_glucose) * 100
    
    # MAGE (Mean Amplitude of Glycemic Excursions)
    # Simplificado: desvio padrão das diferenças
    glucose_diff = np.diff(glucose_values)
    mage = np.std(glucose_diff[np.abs(glucose_diff) > np.std(glucose_values)])
    
    # J-Index (combinação de média e variabilidade)
    j_index = 0.001 * (mean_glucose + np.std(glucose_values))**2
    
    # CONGA (Continuous Overall Net Glycemic Action)
    # Desvio padrão das diferenças a cada hora (6 pontos)
    if len(glucose_values) >= 6:
        hourly_diffs = [glucose_values[i] - glucose_values[i-6] 
                       for i in range(6, len(glucose_values))]
        conga = np.std(hourly_diffs) if hourly_diffs else 0
    else:
        conga = 0
    
    # GRADE (Glycemic Risk Assessment Diabetes Equation)
    # Transformação logarítmica dos valores de glucose
    glucose_mmol = glucose_values / 18.0  # Converter para mmol/L
    log_glucose = np.log(glucose_mmol)
    transformed = 1.509 * (log_glucose**1.084 - 5.381)
    grade = np.mean(transformed**2)
    
    metrics = {
        'GMI (%)': round(gmi, 1),
        'CV (%)': round(cv, 1),
        'MAGE': round(mage, 1),
        'J-Index': round(j_index, 3),
        'CONGA': round(conga, 1),
        'GRADE': round(grade, 2)
    }
    
    return metrics

advanced_metrics = calculate_advanced_metrics(df)

print("📊 MÉTRICAS AVANÇADAS DE QUALIDADE")
print("="*40)
for metric, value in advanced_metrics.items():
    print(f"{metric:15}: {value}")

# Interpretação das métricas
print("\n💡 INTERPRETAÇÃO:")
if advanced_metrics['GMI (%)'] <= 7.0:
    print("✅ GMI excelente (≤7%)")
elif advanced_metrics['GMI (%)'] <= 8.0:
    print("⚠️ GMI moderado (7-8%)")
else:
    print("🚨 GMI elevado (>8%)")

if advanced_metrics['CV (%)'] <= 36:
    print("✅ Baixa variabilidade glicêmica (CV ≤36%)")
else:
    print("⚠️ Alta variabilidade glicêmica (CV >36%)")

# Célula 10: Comparação com benchmarks
def compare_with_benchmarks(df):
    """Compara métricas com benchmarks clínicos"""
    
    # Benchmarks da literatura para sistemas CGM
    benchmarks = {
        'Excelente': {'TIR': 0.80, 'TBR': 0.02, 'TAR': 0.18, 'CV': 30},
        'Bom': {'TIR': 0.70, 'TBR': 0.04, 'TAR': 0.26, 'CV': 36},
        'Moderado': {'TIR': 0.60, 'TBR': 0.06, 'TAR': 0.34, 'CV': 42},
        'Inadequado': {'TIR': 0.50, 'TBR': 0.10, 'TAR': 0.40, 'CV': 50}
    }
    
    # Calcular métricas atuais
    tir = ((df['glucose'] >= 70) & (df['glucose'] <= 180)).mean()
    tbr = (df['glucose'] < 70).mean()
    tar = (df['glucose'] > 180).mean()
    cv = (df['glucose'].std() / df['glucose'].mean()) * 100
    
    current_metrics = {'TIR': tir, 'TBR': tbr, 'TAR': tar, 'CV': cv}
    
    # Determinar classificação
    classification = 'Inadequado'
    for level in ['Excelente', 'Bom', 'Moderado']:
        if (tir >= benchmarks[level]['TIR'] and 
            tbr <= benchmarks[level]['TBR'] and
            cv <= benchmarks[level]['CV']):
            classification = level
            break
    
    print("🎯 COMPARAÇÃO COM BENCHMARKS CLÍNICOS")
    print("="*45)
    
    # Criar tabela de comparação
    comparison_data = []
    for level, bench in benchmarks.items():
        comparison_data.append({
            'Classificação': level,
            'TIR_bench': f"{bench['TIR']:.0%}",
            'TBR_bench': f"{bench['TBR']:.0%}",
            'CV_bench': f"{bench['CV']:.0f}%",
            'Status': '✅' if level == classification else ''
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))
    
    print(f"\n🏆 CLASSIFICAÇÃO ATUAL: {classification}")
    print(f"   TIR: {tir:.0%} | TBR: {tbr:.0%} | CV: {cv:.0f}%")
    
    # Sugestões de melhoria
    print(f"\n💡 SUGESTÕES PARA PRÓXIMO NÍVEL:")
    if classification != 'Excelente':
        next_levels = ['Moderado', 'Bom', 'Excelente']
        try:
            current_idx = next_levels.index(classification)
            next_level = next_levels[current_idx + 1] if current_idx < len(next_levels) - 1 else 'Excelente'
        except ValueError:
            next_level = 'Moderado'
        
        target = benchmarks[next_level]
        
        if tir < target['TIR']:
            print(f"   • Aumentar TIR para {target['TIR']:.0%} (+{(target['TIR']-tir)*100:.1f}pp)")
        if tbr > target['TBR']:
            print(f"   • Reduzir TBR para {target['TBR']:.0%} (-{(tbr-target['TBR'])*100:.1f}pp)")
        if cv > target['CV']:
            print(f"   • Reduzir CV para {target['CV']:.0f}% (-{cv-target['CV']:.1f}pp)")

compare_with_benchmarks(df)

# Célula 11: Relatório executivo automatizado
def generate_executive_report(df):
    """Gera relatório executivo automatizado"""
    
    # Calcular todas as métricas necessárias
    duration_days = (df['timestamp'].max() - df['timestamp'].min()).days
    total_readings = len(df)
    
    # Métricas básicas
    tir = ((df['glucose'] >= 70) & (df['glucose'] <= 180)).mean()
    mean_glucose = df['glucose'].mean()
    cv = (df['glucose'].std() / mean_glucose) * 100
    
    # Eventos adversos
    hypo_events = len([1 for i in range(len(df)-1) 
                      if df.iloc[i]['glucose'] < 70 and df.iloc[i+1]['glucose'] < 70])
    
    # Insulina
    total_insulin = df['insulin_dose'].sum()
    avg_daily_insulin = total_insulin / duration_days if duration_days > 0 else 0
    
    # Confiança do sistema
    avg_confidence = df['confidence'].mean()
    final_confidence = df['confidence'].tail(100).mean()  # Últimas leituras
    
    # Status geral baseado em métricas
    if tir >= 0.7 and cv <= 36 and hypo_events == 0:
        status = "🟢 EXCELENTE"
    elif tir >= 0.6 and cv <= 42:
        status = "🟡 BOM"
    elif tir >= 0.5:
        status = "🟠 MODERADO"
    else:
        status = "🔴 NECESSITA AJUSTES"
    
    report = f"""
╔══════════════════════════════════════════════════════════════════╗
║                  RELATÓRIO EXECUTIVO - SISTEMA APS               ║
╚══════════════════════════════════════════════════════════════════╝

📊 RESUMO GERAL
{'='*50}
Status do Sistema: {status}
Período Analisado: {duration_days} dias ({total_readings:,} leituras)
Última Atualização: {datetime.now().strftime('%d/%m/%Y %H:%M')}

🎯 MÉTRICAS PRINCIPAIS
{'='*50}
• Tempo em Faixa (70-180): {tir:.1%} {'✅' if tir >= 0.7 else '⚠️' if tir >= 0.6 else '🚨'}
• Glicemia Média: {mean_glucose:.0f} mg/dL {'✅' if 100 <= mean_glucose <= 154 else '⚠️'}
• Variabilidade (CV): {cv:.0f}% {'✅' if cv <= 36 else '⚠️' if cv <= 42 else '🚨'}
• Eventos de Hipoglicemia: {hypo_events} {'✅' if hypo_events == 0 else '🚨'}

💉 USO DE INSULINA
{'='*50}
• Total do Período: {total_insulin:.1f} U
• Média Diária: {avg_daily_insulin:.1f} U/dia
• Eficiência: {mean_glucose/avg_daily_insulin if avg_daily_insulin > 0 else 0:.1f} mg/dL por U/dia

🤖 DESEMPENHO DO SISTEMA
{'='*50}
• Confiança Média: {avg_confidence:.1%}
• Confiança Atual: {final_confidence:.1%}
• Evolução: {'📈 Melhorando' if final_confidence > avg_confidence else '📉 Estabilizando'}

🎯 RECOMENDAÇÕES PRIORITÁRIAS
{'='*50}"""

    # Adicionar recomendações baseadas nas métricas
    recommendations = []
    
    if tir < 0.7:
        recommendations.append("• PRIORIDADE ALTA: Melhorar tempo em faixa alvo")
    
    if cv > 36:
        recommendations.append("• Reduzir variabilidade glicêmica - ajustar parâmetros PID")
    
    if hypo_events > 0:
        recommendations.append("• CRÍTICO: Implementar melhor prevenção de hipoglicemia")
    
    if avg_daily_insulin > 100:
        recommendations.append("• Revisar configurações de sensibilidade à insulina")
    
    if final_confidence < 0.8:
        recommendations.append("• Continuar período de adaptação do sistema ML")
    
    if not recommendations:
        recommendations.append("• Sistema operando dentro dos parâmetros adequados")
        recommendations.append("• Manter monitoramento e ajustes finos")
    
    for rec in recommendations:
        report += f"\n{rec}"
    
    report += f"""

📈 PRÓXIMOS PASSOS
{'='*50}
1. Análise detalhada dos padrões circadianos
2. Otimização dos parâmetros do controlador híbrido
3. Validação em cenários de estresse (refeições grandes, exercícios)
4. Comparação com sistemas APS comerciais

⚠️  DISCLAIMER
{'='*50}
Este relatório é baseado em dados de simulação para fins de pesquisa.
NÃO utilizar para decisões clínicas sem validação médica adequada.

Gerado automaticamente pelo Sistema APS v1.0
"""
    
    return report

# Gerar e exibir relatório
executive_report = generate_executive_report(df)
print(executive_report)

# Célula 12: Exportação de dados e gráficos
def export_analysis_results(df, stats, advanced_metrics):
    """Exporta resultados da análise"""
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Exportar dataset processado
    df_export = df.copy()
    df_export['date'] = df_export['timestamp'].dt.date
    df_export['time'] = df_export['timestamp'].dt.time
    
    filename_data = f'analysis_data_{timestamp}.csv'
    df_export.to_csv(filename_data, index=False)
    print(f"📁 Dataset exportado: {filename_data}")
    
    # Exportar métricas consolidadas
    metrics_summary = {
        **stats,
        **advanced_metrics,
        'analysis_timestamp': datetime.now().isoformat(),
        'data_period_days': (df['timestamp'].max() - df['timestamp'].min()).days,
        'total_readings': len(df)
    }
    
    filename_metrics = f'metrics_summary_{timestamp}.json'
    import json
    with open(filename_metrics, 'w') as f:
        json.dump(metrics_summary, f, indent=2, default=str)
    print(f"📊 Métricas exportadas: {filename_metrics}")
    
    # Criar relatório HTML
    html_report = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Relatório de Análise APS</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            .metric {{ margin: 10px 0; padding: 10px; border-left: 4px solid #007bff; }}
            .excellent {{ border-left-color: #28a745; }}
            .warning {{ border-left-color: #ffc107; }}
            .danger {{ border-left-color: #dc3545; }}
            pre {{ background: #f8f9fa; padding: 15px; border-radius: 5px; }}
        </style>
    </head>
    <body>
        <h1>Relatório de Análise - Sistema de Pâncreas Artificial</h1>
        <p>Gerado em: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}</p>
        
        <h2>Resumo Executivo</h2>
        <pre>{executive_report}</pre>
        
        <h2>Métricas Detalhadas</h2>
        <div class="metric {'excellent' if stats['time_in_range'] >= 0.7 else 'warning' if stats['time_in_range'] >= 0.6 else 'danger'}">
            <strong>Tempo em Faixa:</strong> {stats['time_in_range']:.1%}
        </div>
        <div class="metric">
            <strong>Glicemia Média:</strong> {stats['glucose_mean']:.1f} mg/dL
        </div>
        <div class="metric {'excellent' if advanced_metrics['CV (%)'] <= 36 else 'warning'}">
            <strong>Coeficiente de Variação:</strong> {advanced_metrics['CV (%)']}%
        </div>
        <div class="metric">
            <strong>GMI Estimado:</strong> {advanced_metrics['GMI (%)']}%
        </div>
        
        <h2>Análise de Eventos</h2>
        <p>Hipoglicemias detectadas: {len(hypo_events)}</p>
        <p>Hiperglicemias detectadas: {len(hyper_events)}</p>
        
        <p><em>Relatório gerado automaticamente pelo sistema APS para fins de pesquisa.</em></p>
    </body>
    </html>
    """
    
    filename_html = f'aps_report_{timestamp}.html'
    with open(filename_html, 'w', encoding='utf-8') as f:
        f.write(html_report)
    print(f"📄 Relatório HTML: {filename_html}")
    
    return {
        'data_file': filename_data,
        'metrics_file': filename_metrics,
        'html_report': filename_html
    }

# Exportar resultados
exported_files = export_analysis_results(df, stats, advanced_metrics)

print(f"""
✅ ANÁLISE CONCLUÍDA COM SUCESSO!

📁 Arquivos gerados:
   • Dados: {exported_files['data_file']}
   • Métricas: {exported_files['metrics_file']}  
   • Relatório: {exported_files['html_report']}

🔬 Para análises avançadas:
   • Use os gráficos interativos acima
   • Examine padrões circadianos
   • Analise correlações entre variáveis
   • Compare diferentes períodos

⚠️  LEMBRETE: Sistema experimental - apenas para pesquisa!
""")

# Célula final: Instruções para próximos passos
print("""
🚀 PRÓXIMOS PASSOS SUGERIDOS:

1. ANÁLISE TEMPORAL
   • Examine tendências semanais e mensais
   • Identifique padrões sazonais
   • Analise impacto de mudanças de configuração

2. OTIMIZAÇÃO DO CONTROLADOR
   • Ajuste parâmetros PID baseado nas métricas
   • Treine modelo ML com mais dados
   • Teste diferentes estratégias de controle

3. VALIDAÇÃO CLÍNICA
   • Compare com dados reais de CGM
   • Valide algoritmos com especialistas
   • Teste em cenários clínicos diversos

4. MELHORIAS DO SISTEMA
   • Implemente detecção automática de refeições
   • Adicione predição de exercícios
   • Desenvolva alertas inteligentes

5. DOCUMENTAÇÃO
   • Documente achados e insights
   • Publique resultados (com disclaimers)
   • Compartilhe com comunidade de pesquisa

Para mais informações, consulte a documentação completa
do projeto em DESIGN.md e README.md
""")

# Script adicional: setup.py para instalação
SETUP_PY_CONTENT = '''
"""
Setup script for Artificial Pancreas System (APS)

Para instalar:
    pip install -e .

Para desenvolvimento:
    pip install -e ".[dev]"
"""

from setuptools import setup, find_packages

with open("README.md", "r", encoding="utf-8") as fh:
    long_description = fh.read()

with open("requirements.txt", "r", encoding="utf-8") as fh:
    requirements = [line.strip() for line in fh if line.strip() and not line.startswith("#")]

setup(
    name="aps-system",
    version="1.0.0",
    author="Sistema APS",
    author_email="aps@example.com",
    description="Sistema de Pâncreas Artificial para Simulação e Pesquisa",
    long_description=long_description,
    long_description_content_type="text/markdown",
    url="https://github.com/example/aps-system",
    packages=find_packages(where="src"),
    package_dir={"": "src"},
    classifiers=[
        "Development Status :: 4 - Beta",
        "Intended Audience :: Science/Research",
        "License :: OSI Approved :: MIT License",
        "Operating System :: OS Independent",
        "Programming Language :: Python :: 3",
        "Programming Language :: Python :: 3.10",
        "Programming Language :: Python :: 3.11",
        "Topic :: Scientific/Engineering :: Medical Science Apps.",
    ],
    python_requires=">=3.10",
    install_requires=requirements,
    extras_require={
        "dev": [
            "pytest>=7.0.0",
            "pytest-asyncio>=0.20.0",
            "black>=22.0.0",
            "flake8>=5.0.0",
            "mypy>=0.990",
            "jupyter>=1.0.0",
            "notebook>=6.5.0",
        ]
    },
    entry_points={
        "console_scripts": [
            "aps-system=main:main",
            "aps-simulate=examples.run_scenarios:main",
            "aps-analyze=notebooks.analysis_example:main",
        ],
    },
    include_package_data=True,
    package_data={
        "": ["*.yaml", "*.json", "*.html", "*.css", "*.js"],
    },
    zip_safe=False,
)
'''

# Salvar setup.py
with open('setup.py', 'w', encoding='utf-8') as f:
    f.write(SETUP_PY_CONTENT)

print("📦 setup.py criado para instalação do pacote")

# Script final: Makefile para automação
MAKEFILE_CONTENT = '''
# Makefile para Sistema de Pâncreas Artificial

.PHONY: help install install-dev test lint format run-simulation run-tests clean docs

help:
	@echo "Sistema de Pâncreas Artificial - Comandos disponíveis:"
	@echo ""
	@echo "  install        Instalar dependências básicas"
	@echo "  install-dev    Instalar dependências de desenvolvimento"
	@echo "  test           Executar todos os testes"
	@echo "  lint           Verificar código com linters"
	@echo "  format         Formatar código"
	@echo "  run-simulation Executar simulação de exemplo"
	@echo "  run-scenarios  Executar cenários de teste"
	@echo "  analyze        Executar análise de dados"
	@echo "  clean          Limpar arquivos temporários"
	@echo "  docs           Gerar documentação"

install:
	pip install -r requirements.txt

install-dev:
	pip install -r requirements.txt
	pip install -e ".[dev]"

test:
	pytest tests/ -v --cov=src --cov-report=html

lint:
	flake8 src/ tests/
	mypy src/

format:
	black src/ tests/ examples/
	isort src/ tests/ examples/

run-simulation:
	python src/main.py --mode simulation --duration 24 --acceleration 10

run-scenarios:
	python examples/run_scenarios.py

analyze:
	jupyter notebook notebooks/analysis_example.ipynb

clean:
	find . -type f -name "*.pyc" -delete
	find . -type d -name "__pycache__" -delete
	rm -rf htmlcov/
	rm -rf .pytest_cache/
	rm -rf .mypy_cache/
	rm -rf logs/*.log
	rm -rf exports/*.csv

docs:
	@echo "Documentação disponível em:"
	@echo "  README.md - Guia principal"
	@echo "  DESIGN.md - Decisões técnicas"
	@echo "  docs/ - Documentação detalhada"

docker-build:
	docker build -t aps-system .

docker-run:
	docker run -p 8000:8000 aps-system

# Comandos de segurança
security-check:
	@echo "⚠️  VERIFICAÇÃO DE SEGURANÇA:"
	@echo "   • Sistema apenas para simulação"
	@echo "   • NÃO usar em pacientes reais" 
	@echo "   • Validação clínica necessária"
	@echo "   • Aprovação regulatória obrigatória"

# Comandos de backup
backup:
	mkdir -p backups/
	tar -czf backups/aps_backup_$(shell date +%Y%m%d_%H%M%S).tar.gz \
		src/ tests/ examples/ notebooks/ config.yaml requirements.txt
	@echo "Backup criado em backups/"

# Comando para publicação (apenas após validação)
publish-research:
	@echo "📚 ANTES DE PUBLICAR:"
	@echo "   ✅ Validar todos os algoritmos"
	@echo "   ✅ Revisar disclaimers de segurança"
	@echo "   ✅ Incluir limitações do estudo"
	@echo "   ✅ Obter aprovação de comitê de ética"
	@echo ""
	@echo "Para prosseguir com publicação de pesquisa:"
	@echo "   make security-check && python scripts/prepare_publication.py"
'''

# Salvar Makefile
with open('Makefile', 'w') as f:
    f.write(MAKEFILE_CONTENT)

print("🔧 Makefile criado para automação do projeto")

print("""
🎉 SISTEMA DE PÂNCREAS ARTIFICIAL COMPLETO!

O projeto foi criado com sucesso incluindo:

📁 ESTRUTURA COMPLETA:
   ✅ Simulador fisiológico baseado UVA/Padova
   ✅ Controlador PID + Machine Learning híbrido
   ✅ Sistema de segurança multicamadas
   ✅ Interface web interativa
   ✅ API REST e WebSocket
   ✅ Métricas avançadas de desempenho
   ✅ Testes automatizados
   ✅ Análise de dados em Jupyter
   ✅ Documentação completa
   ✅ Scripts de exemplo e cenários

🚀 PARA COMEÇAR:
   1. pip install -r requirements.txt
   2. python src/main.py --mode simulation --web
   3. Abrir http://localhost:8000
   4. Explorar cenários em examples/run_scenarios.py

⚠️  IMPORTANTE:
   • Sistema EXPERIMENTAL apenas
   • NÃO usar clinicamente
   • Requer validação e aprovação
   • Consultar README.md para detalhes

🔬 CONTRIBUIÇÕES:
   • Testar algoritmos
   • Melhorar métricas
   • Adicionar cenários
   • Validar com dados reais
   • Otimizar performance

Boa pesquisa e desenvolvimento responsável! 🧬👨‍⚕️👩‍⚕️
""")

# Comentário final com instruções de uso do notebook
"""
INSTRUÇÕES PARA USO DESTE NOTEBOOK:

1. CONFIGURAÇÃO INICIAL:
   - Instalar dependências: pip install -r requirements.txt
   - Verificar dados disponíveis ou usar dados sintéticos
   - Configurar parâmetros no início do notebook

2. EXECUÇÃO:
   - Executar células sequencialmente
   - Modificar parâmetros conforme necessário  
   - Salvar gráficos e resultados importantes

3. ANÁLISE CUSTOMIZADA:
   - Adaptar funções para seus dados específicos
   - Adicionar métricas personalizadas
   - Criar visualizações específicas

4. EXPORTAÇÃO:
   - Usar funções de exportação para relatórios
   - Salvar gráficos em alta resolução
   - Gerar relatórios HTML para compartilhamento

5. PRÓXIMOS PASSOS:
   - Integrar com pipeline de ML
   - Automatizar análises periódicas
   - Criar dashboards em tempo real

⚠️ LEMBRETE IMPORTANTE:
Este notebook é para análise de dados de SIMULAÇÃO apenas.
Nunca usar resultados para decisões clínicas reais sem
validação médica adequada e aprovação regulatória.
"""# notebooks/analysis_example.ipynb - Notebook de Análise
"""
Notebook Jupyter para análise avançada do Sistema de Pâncreas Artificial

Este notebook demonstra:
1. Carregamento e preprocessamento de dados de simulação
2. Análise de métricas de desempenho
3. Visualizações interativas
4. Comparações entre diferentes configurações
5. Análise de padrões temporais
6. Relatórios automatizados
"""

# Célula 1: Imports e configuração
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.express as px
from plotly.sub